In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import glob
import random
from google.colab import drive
from PIL import Image
from torch.nn import functional as F
import heapq

def gen_split(root_dir, stackSize, train, frameType):
    Dataset = []
    Labels = []
    NumFrames = []
    if train==True:
      list_folder=['S1', 'S3', 'S4']
    elif train==False:
      list_folder=['S2']
    for dir_user in list_folder: #S1,S2,S3,S4
      class_id=0
      dir = os.path.join(root_dir, dir_user)
      for target in sorted(os.listdir(dir)): #action1, action2
        dir1 = os.path.join(dir, target)
        insts = sorted(os.listdir(dir1))
        if insts != []:
          i=0
          for inst in insts:
            if frameType=='r':
              inst_dir = os.path.join(dir1, inst, 'rgb')
            elif frameType=='m':
              inst_dir = os.path.join(dir1, inst, 'mmaps')
            numFrames = len(glob.glob1(inst_dir, '*.png'))
            if numFrames >= stackSize or inst_dir=='/content/gdrive/My Drive/Project FPAR2/ego-rnn/GTEA61/processed_frames2/S3/stir_spoon,cup/2/mmaps' or inst_dir=='/content/gdrive/My Drive/Project FPAR2/ego-rnn/GTEA61/processed_frames2/S4/pour_sugar,spoon,cup/1/mmaps':
              Dataset.append(inst_dir)
              Labels.append(class_id)
              NumFrames.append(numFrames)
            else:
              i+=1
          if len(insts)==i:
            print(f'This should not be considered! Frametype: {frametype}')
            print(dir1)
        class_id+=1
    return Dataset, Labels, NumFrames   

class makeDataset(Dataset):

    def __init__(self, root_dir, spatial_transform=None, seqLen=20,
                 train=True, mulSeg=False, numSeg=1, fmt='.png', frameType='m'):

        self.frameType=frameType
        self.root_dir=root_dir
        self.images, self.labels, self.numFrames = gen_split(root_dir, 5, train, frameType)
        self.spatial_transform = spatial_transform
        self.train = train
        self.mulSeg = mulSeg
        self.numSeg = numSeg
        self.seqLen = seqLen
        self.fmt = fmt

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        vid_name = self.images[idx]
        label = self.labels[idx]
        numFrame = self.numFrames[idx]
        inpSeq = []
        black=0
        tot=0
        back=0
        seqLen=self.seqLen
        self.spatial_transform.randomize_parameters()
        dic={}
        n=max(seqLen, numFrame-1)

        for i in np.linspace(1, numFrame, n, endpoint=False):
        #np.linspace(1, numFrame, 1, endpoint=True):
          tot+=1

          try:
            fl_name = vid_name + '/' + 'map' + str(int(np.floor(i))).zfill(4) + self.fmt
            img = Image.open(fl_name)
          except:
            try:
              fl_name = vid_name + '/' + 'map' + str(int(np.floor(i+1))).zfill(4) + self.fmt
              img = Image.open(fl_name)
            except:
              try:
                fl_name = vid_name + '/' + 'map' + str(int(np.floor(i-1))).zfill(4) + self.fmt
                img = Image.open(fl_name)
              except:
                try:
                  fl_name = vid_name + '/' + 'map' + str(int(np.floor(i+2))).zfill(4) + self.fmt
                  img = Image.open(fl_name)
                except:
                  try:
                    fl_name = vid_name + '/' + 'map' + str(int(np.floor(i-2))).zfill(4) + self.fmt
                    img = Image.open(fl_name)
                  except:
                    a=0
          img=self.spatial_transform(img.convert('L'))
          summ=torch.sum(img)
          dic[np.floor(i)]=summ
        #h=sorted(heapq.nlargest(seqLen, dic, key=dic.get))
        h=sorted(dic, key=dic.get, reverse=True)[:seqLen]
        h=sorted(h)
        #print(len(h))
        #print(vid_name, h)
        for i in h:
          fl_name = vid_name + '/' + 'rgb' + str(int(np.floor(i))).zfill(4) + self.fmt
          f_name=fl_name.replace('mmaps','rgb')
          img = Image.open(f_name)
          img=self.spatial_transform(img.convert('RGB'))
          inpSeq.append(img)
        if len(inpSeq)!=seqLen:
          diff=seqLen-len(inpSeq)
          for p in range(diff):
            inpSeq.append(inpSeq[-1])              
        inpSeq = torch.stack(inpSeq, 0)
        return inpSeq, label

In [2]:
'''
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/gdrive',  force_remount=True)
%cd /content/gdrive/My Drive/Project FPAR2/ego-rnn

from spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)

normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
spatial_transform = Compose([Scale(256), RandomHorizontalFlip(), MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224), ToTensor(), normalize])
from torchvision.transforms import Resize
spatial_transform_mmap = Compose([Resize(256),ToTensor()])
seqLen=7
train_data_dir='./GTEA61/processed_frames2'
vid_seq_train = makeDataset(train_data_dir,
                            spatial_transform=spatial_transform, seqLen=seqLen, frameType='m', fmt='.png', train=False)
trainBatchSize=32
train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize,
                        shuffle=True, num_workers=4, pin_memory=True)
'''

16
